In [11]:
#step1 Reading file and understand data
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('salary_prediction').getOrCreate()
df=spark.read.csv(r"C:\Users\Vamsi\Downloads\test2.csv",header=True,inferSchema=True)
df=df.na.drop()
df.show()
df.describe().show()
df.printSchema()

+---------+---+----------+------+
|     Name|age|Experience|Salary|
+---------+---+----------+------+
|    Krish| 31|        10| 30000|
|Sudhanshu| 30|         8| 25000|
|    Sunny| 29|         4| 20000|
|     Paul| 24|         3| 20000|
|   Harsha| 21|         1| 15000|
|  Shubham| 23|         2| 18000|
+---------+---+----------+------+

+-------+------+------------------+-----------------+------------------+
|summary|  Name|               age|       Experience|            Salary|
+-------+------+------------------+-----------------+------------------+
|  count|     6|                 6|                6|                 6|
|   mean|  NULL|26.333333333333332|4.666666666666667|21333.333333333332|
| stddev|  NULL| 4.179314138308661|3.559026084010437| 5354.126134736337|
|    min|Harsha|                21|                1|             15000|
|    max| Sunny|                31|               10|             30000|
+-------+------+------------------+-----------------+------------------+

r

In [12]:
#Combine multiple features into single feature
from pyspark.ml.feature import VectorAssembler
feacturecols=['age', 'Experience']
assembler=VectorAssembler(inputCols=feacturecols,outputCol='features')
final_data=assembler.transform(df)
final_data.show()

+---------+---+----------+------+-----------+
|     Name|age|Experience|Salary|   features|
+---------+---+----------+------+-----------+
|    Krish| 31|        10| 30000|[31.0,10.0]|
|Sudhanshu| 30|         8| 25000| [30.0,8.0]|
|    Sunny| 29|         4| 20000| [29.0,4.0]|
|     Paul| 24|         3| 20000| [24.0,3.0]|
|   Harsha| 21|         1| 15000| [21.0,1.0]|
|  Shubham| 23|         2| 18000| [23.0,2.0]|
+---------+---+----------+------+-----------+



In [19]:
#Split the data into two 1)train_data 2)test_data
train_data, test_data = final_data.randomSplit([0.80,0.20])
train_data.count()
test_data.count()

2

In [20]:
#Train the model
from pyspark.ml.regression import LinearRegression
lr=LinearRegression(featuresCol='features', labelCol='Salary')
lr_model=lr.fit(train_data)

In [26]:
#Evaluate the model
print('coefficients:',lr_model.coefficients)
print('Intercept:',lr_model.intercept)
test_result=lr_model.evaluate(test_data)
print("RMSE:", test_result.rootMeanSquaredError)
print("R2 Score:", test_result.r2)

coefficients: [47.61904761904503,1285.7142857142912]
Intercept: 13619.047619047662
RMSE: 1533.4516369623352
R2 Score: 0.934681279919376


In [22]:
#Make predictions
prediction=lr_model.transform(test_data)
prediction.show()

+-------+---+----------+------+-----------+-----------------+
|   Name|age|Experience|Salary|   features|       prediction|
+-------+---+----------+------+-----------+-----------------+
|  Krish| 31|        10| 30000|[31.0,10.0]|27952.38095238097|
|Shubham| 23|         2| 18000| [23.0,2.0]|17285.71428571428|
+-------+---+----------+------+-----------+-----------------+



new input file for trained model

In [23]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('New_data').getOrCreate()
df1=spark.read.csv(r"C:\Users\Vamsi\Downloads\Book2.csv",header=True,inferSchema=True)
df_final=df1.drop('Salary')
df_final.show()
df_final.columns

+-----+---+----------+
| Name|age|Experience|
+-----+---+----------+
|vamsi| 34|        10|
+-----+---+----------+



['Name', 'age', 'Experience']

In [31]:
from pyspark.ml.feature import VectorAssembler
new_df=VectorAssembler(inputCols=['age', 'Experience'],outputCol='features')
final_data1=new_df.transform(df_final)
predict=lr_model.transform(final_data1)
predict.show()

+-----+---+----------+-----------+------------------+
| Name|age|Experience|   features|        prediction|
+-----+---+----------+-----------+------------------+
|vamsi| 34|        10|[34.0,10.0]|28095.238095238106|
+-----+---+----------+-----------+------------------+

